In [10]:
from PIL import Image
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [24]:
def resize_images(img_path):
    
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")

# 가위, 바위, 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
scissor1_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/scissor1"
rock1_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/rock1"
paper1_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/paper1"

resize_images(scissor1_image_dir_path)
resize_images(rock1_image_dir_path)
resize_images(paper1_image_dir_path)

print("가위 이미지 resize 완료!")
print("바위 이미지 resize 완료!")
print("보 이미지 resize 완료!")

100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!


In [25]:
def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


In [26]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

Model에 추가된 Layer 개수:  7


In [27]:
# model 생성
tf.random.set_seed(1234)  # 동일한 결과를 보장하기 위해 seed 고정
model=keras.models.Sequential() # ↓사용할 이미지 특징의 수        # ↓(28x28픽셀)x3(RGB)
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu')) # 사용할 이미지 특징의 수
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))    # 분류기에 사용될 뉴런의 수(알고리즘의 복잡도)
model.add(keras.layers.Dense(3, activation='softmax'))  # 3개의 class로 분류

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

# model 학습시키기
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=10)  # epochs=학습 반복 횟수

Model에 추가된 Layer 개수:  7
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                102464    
_________________________________________________________________
dense_5 (Dense)              (

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                102464    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                

In [15]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train_norm, y_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 5s 187ms/step - loss: 1.0852 - accuracy: 0.3967
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 1.0020 - accuracy: 0.7272
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 0.8663 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 4ms/step - loss: 0.6733 - accuracy: 0.9938
Epoch 5/10
10/10 [==============================] - 0s 4ms/step - loss: 0.4405 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2389 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 4ms/step - loss: 0.1138 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0470 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0240 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0152 - accuracy: 1.0000


In [29]:
test_scissor1_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/test/scissor1"
resize_images(test_scissor1_image_dir_path)   # 테스트 가위 이미지 resize

test_rock1_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/test/rock1"
resize_images(test_rock1_image_dir_path)   # 테스트 바위 이미지 resize

test_paper1_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/test/paper1"
resize_images(test_paper1_image_dir_path)   # 테스트 보 이미지 resize

100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.


In [30]:
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper1/test"
(x_test, y_test)=load_data(image_dir_path)

print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


In [31]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose = 2)

print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

10/10 - 0s - loss: 319.8391 - accuracy: 0.4500
test_loss : 319.8390808105469
test_accuracy : 0.44999998807907104
